# Pima Data cleaning



In [680]:
# Import Modules Here
import pandas as pd

## Part 1: Read the Data into Jupyter

In [682]:
# Read the data into a dataframe named pima_df.
# YOUR CODE HERE
pima_df = pd.read_csv('diabetes_uncleaned_data.csv')

## Part 2: Missing Data

In [684]:
#Question 1:  Find Columns with missing data
# YOUR CODE HERE
pima_df.isna().sum()

Pregnancies                 114
Glucose                      16
BloodPressure                22
SkinThickness                 0
Insulin                       0
BMI                           1
DiabetesPedigreeFunction      0
Age                          22
Outcome                       0
dtype: int64

In [685]:
# Question 2:  Write code to remove rows with missing data and save the new dataframe as pima_missing_fixed_df
# YOUR CODE HERE
pima_missing_fixed_df = pima_df.dropna()
pima_missing_fixed_df.isna().sum()

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

In [686]:
# Question 3:  Print out the number of rows in the dataframe.
# YOUR CODE HERE
len(pima_missing_fixed_df)
pima_missing_fixed_df.shape[0]

840

### Question 4:  Discuss why missing data could be a problem.  What other methods could you use in this situation?

Missing data could cause our information to be less accurate, meaning that we are missing big pieces of the puzzle. We could ignore the null values instead of deleting them or replace the null values.

## Part 3: Duplicated Data

In [689]:
# Question 1: find the rows of duplicated data in the
# pima_missing_fixed_df and print them.
pima_missing_fixed_df.duplicated()

0       False
1       False
2       False
4       False
5       False
        ...  
998     False
999      True
1001    False
1002    False
1003     True
Length: 840, dtype: bool

In [690]:
# Question 2: Write code to remove duplicate rows
# and save the new dataframe as pima_missing_fixed_df
pima_missing_fixed_df = pima_missing_fixed_df.drop_duplicates()

In [691]:
# Question 3:  Print out the number of rows in the dataframe.
pima_missing_fixed_df.shape[0]

836

### Question 4:  In your jupyter notebook, discuss why duplicate data could be a problem.
### What other methods could you use in this situation?

It can lead to bad calculations show the wrong picture when you go to present the data.
Instead of removing all duplicates we can determine what is and what is not a duplicate.
    ex) .drop_duplicates(subset=['columns to check for duplicates'],keep='first')

## Part 4: Mis-typed Columns/Broken Data


In [694]:
# Question 1: find the columns where most of the data seems to be one type of data, and there seems to be a data error. In the markdown, after exploring in code, add a markdown cell discussing which columns have the data error.
print(pima_missing_fixed_df['BMI'].value_counts())
print(pima_missing_fixed_df.info())

BMI
7.0     6
17.4    6
59.6    6
15.1    6
34.4    5
       ..
33.7    1
38.2    1
1.7     1
3.6     1
4.8     1
Name: count, Length: 451, dtype: int64
<class 'pandas.core.frame.DataFrame'>
Index: 836 entries, 0 to 1002
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               836 non-null    float64
 1   Glucose                   836 non-null    float64
 2   BloodPressure             836 non-null    object 
 3   SkinThickness             836 non-null    int64  
 4   Insulin                   836 non-null    int64  
 5   BMI                       836 non-null    object 
 6   DiabetesPedigreeFunction  836 non-null    float64
 7   Age                       836 non-null    float64
 8   Outcome                   836 non-null    object 
dtypes: float64(4), int64(2), object(3)
memory usage: 65.3+ KB
None


BloodPressure: 2 Errors, BMI: Error, Outcome: 1 ERROR

In [696]:
# Question 2: Remove the rows with data errors
pima_missing_fixed_df = pima_missing_fixed_df.loc[pima_missing_fixed_df['Outcome']!= 'ERROR', :]
pima_missing_fixed_df = pima_missing_fixed_df.loc[pima_missing_fixed_df['BMI']!= 'Error', :]
pima_missing_fixed_df = pima_missing_fixed_df.loc[pima_missing_fixed_df['BloodPressure']!= 'Error', :]

print(pima_missing_fixed_df['BloodPressure'].value_counts())

BloodPressure
26       15
27       15
82       15
96       15
81       13
         ..
104       4
107       4
13        3
10000     2
19        1
Name: count, Length: 112, dtype: int64


In [697]:
# Question 3: After the data is fixed in your columns change the columns to the correct type and save this as the pima_fixed_columns_df.
def word_bool(x: str):
    if x.lower() == 'false':
        return False
    elif x.lower() == 'true':
        return True
    else:
        raise Exception("This must be True or False case insensitive")

pima_missing_fixed_df['Outcome'] = pima_missing_fixed_df['Outcome'].apply(word_bool)
pima_missing_fixed_df['Outcome'].value_counts()

Outcome
False    441
True     391
Name: count, dtype: int64

In [698]:
# Question 4: Run pima_fixed_columns_df.info() to confirm the columns have changed.
pima_missing_fixed_df['BMI'] = pima_missing_fixed_df['BMI'].astype('float64')
pima_missing_fixed_df['BloodPressure'] = pima_missing_fixed_df['BloodPressure'].astype('float')
pima_missing_fixed_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 832 entries, 0 to 1001
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               832 non-null    float64
 1   Glucose                   832 non-null    float64
 2   BloodPressure             832 non-null    float64
 3   SkinThickness             832 non-null    int64  
 4   Insulin                   832 non-null    int64  
 5   BMI                       832 non-null    float64
 6   DiabetesPedigreeFunction  832 non-null    float64
 7   Age                       832 non-null    float64
 8   Outcome                   832 non-null    bool   
dtypes: bool(1), float64(6), int64(2)
memory usage: 59.3 KB


## Part 5: Outlier Detection and Removal


In [723]:
# Question 1: Print out the Outliers in each column in the pima_fixed_columns_df dataframe, use the IQR method of outlier detection.
def is_outlier(column: pd.Series):
        Q1 = column.quantile(0.25)
        Q3 = column.quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        return ~column.between(lower_bound, upper_bound)
    
display(pima_missing_fixed_df[is_outlier(pima_missing_fixed_df['Pregnancies'])])
display(pima_missing_fixed_df[is_outlier(pima_missing_fixed_df['Glucose'])])
display(pima_missing_fixed_df[is_outlier(pima_missing_fixed_df['BloodPressure'])])
display(pima_missing_fixed_df[is_outlier(pima_missing_fixed_df['SkinThickness'])])
display(pima_missing_fixed_df[is_outlier(pima_missing_fixed_df['Insulin'])])
display(pima_missing_fixed_df[is_outlier(pima_missing_fixed_df['BMI'])])
display(pima_missing_fixed_df[is_outlier(pima_missing_fixed_df['DiabetesPedigreeFunction'])])
display(pima_missing_fixed_df[is_outlier(pima_missing_fixed_df['Age'])])
# pregnancy, bloodpressure, and insulin have outliers

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
5,-100.0,142.0,88.0,39,82,42.7,1.369,148.0,True


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
649,3.0,155.0,10000.0,35,748,32.6,0.098,85.0,True
995,5.0,62.0,10000.0,13,412,48.7,0.267,101.0,False


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
1,13.0,151.0,107.0,38,10000,9.2,0.507,148.0,False


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome


In [731]:
# Question 2: Use loc to remove outliers in each of the columns that have outliers, save this as pima_outlier_removed_df.

pima_outlier_removed_df = pima_missing_fixed_df[is_outlier(pima_missing_fixed_df['Pregnancies']) != True]
pima_outlier_removed_df = pima_outlier_removed_df[is_outlier(pima_outlier_removed_df['BloodPressure']) != True]
pima_outlier_removed_df = pima_outlier_removed_df[is_outlier(pima_outlier_removed_df['Insulin']) != True]


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,3.0,167.0,39.0,28,545,53.0,2.447,125.0,False
2,8.0,59.0,109.0,18,739,59.1,1.769,91.0,True
4,10.0,165.0,48.0,37,737,39.4,0.729,73.0,False
6,6.0,151.0,60.0,26,362,43.4,2.027,23.0,False
7,0.0,179.0,0.0,19,238,47.7,2.656,77.0,False


In [735]:
# Question 3:  Print out the row count in the pima_outlier_removed_df and confirm this number is correct.
print(f"{pima_missing_fixed_df.shape[0]} is the numbre of rows before correcting for outliers")
print(f"{pima_outlier_removed_df.shape[0]} is the number of rows left after corrections")

832 is the numbre of rows before correcting for outliers
828 is the number of rows left after corrections
